In [1]:
import pandas as pd
import numpy as np
import os

In [83]:
jira_issues = pd.DataFrame()
DATAPATH = "jira_defects/"

In [84]:
for fil in os.listdir('jira_defects'):
    if fil.endswith("csv"):
        jira_data = pd.read_csv(DATAPATH+fil)
    elif fil.endswith("xlsx"):
        jira_data = pd.read_excel(DATAPATH+fil)
    jira_data['Remove_flag_v2'] = jira_data['Issue key'].str.match('\w+-\d+')
    jira_data = jira_data[jira_data['Remove_flag_v2']==True]
    custom_cols = [x for x in jira_data.columns.tolist() if 'Custom field' in x]
    outward_cols = [x for x in jira_data.columns.tolist() if 'Outward issue link' in x]
    jira_data.drop(custom_cols, axis=1, inplace=True)
    jira_data.drop(outward_cols, axis=1, inplace=True)
    jira_issues = jira_issues.append(jira_data, ignore_index=True)

C:\Users\suresh.nagulavancha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,229,234,243,244,251,252,253,255,259,264,276,280,288,291,297,298,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,3

In [4]:
jira_issues = jira_issues.drop_duplicates()

In [5]:
jira_issues = jira_issues.groupby('Issue key').first().reset_index()

In [6]:
jira_issues.drop(['Remove_flag_v2'], axis=1, inplace=True)

In [7]:
jira_issues.shape

(20794, 1301)

In [8]:
null_percent = pd.DataFrame(jira_issues.isnull().sum()/jira_issues.shape[0]).reset_index()
null_percent.columns = ['Column',"Percentage"]

In [9]:
null_percent['To_remove'] = null_percent['Percentage']>0.999

In [10]:
null_columns = null_percent[null_percent['To_remove']==True]['Column'].tolist()

In [11]:
jira_issues.drop(null_columns, axis=1, inplace=True)

In [12]:
cols_to_drop = ['Project lead','Project name','Project type','Project url','Priority','Assignee','Reporter','Creator',\
               'Created','Updated','Last Viewed','Due Date','Votes','Watchers','Original Estimate','Remaining Estimate',\
               'Time Spent','Work Ratio','Σ Original Estimate','Σ Remaining Estimate', 'Σ Time Spent','Security Level',\
                'Watchers.1', 'Watchers.2', 'Watchers.3', 'Watchers.4', 'Watchers.5','Watchers.6', 'Watchers.7','Watchers.8',\
                'Watchers.9','Watchers.10', 'Watchers.11', 'Watchers.12', 'Watchers.13','Watchers.14','Watchers.15',\
                'Watchers.16', 'Watchers.17','Watchers.18', 'Watchers.19', 'Watchers.20','Sprint','Parent id','Resolved','Issue id']

In [13]:
jira_issues.drop(cols_to_drop, axis=1, inplace=True)

In [137]:
log_columns = [x for x in jira_issues.columns.tolist() if x.startswith("Log Work")]
attach_columns = [x for x in jira_issues.columns.tolist() if x.startswith("Attachment")]
comments_columns = [x for x in jira_issues.columns.tolist() if x.startswith("Comment")]
labels_column = [x for x in jira_issues.columns.tolist() if x.startswith("Labels")]

In [56]:
jira_issues['Log_Text_Complete'] = jira_issues[log_columns].apply(lambda row:"_".join(row.values.astype(str)), axis=1)
jira_issues['attach_complete'] = jira_issues[attach_columns].apply(lambda row:"_".join(row.values.astype(str)), axis=1)
jira_issues['comments_complete'] = jira_issues[comments_columns].apply(lambda row:"_".join(row.values.astype(str)), axis=1)
jira_issues['Labels_complete'] = jira_issues[labels_column].apply(lambda row:"_".join(row.values.astype(str)), axis=1)

In [103]:
jira_issues['Log_text_extract'] = jira_issues['Log_Text_Complete'].apply(lambda text: 1 if "pull request" in text.lower() else 0)

In [104]:
jira_issues['attachment_extract'] = jira_issues['attach_complete'].apply(lambda text: 1 if "patch" in text.lower() or "diff" in text.lower() else 0)

In [105]:
jira_issues['comments_extract'] = jira_issues['comments_complete'].apply(lambda text: 1 if "commit" in text.lower() or "pull" in text.lower() or "code changes" in text.lower() or "code changed" in text.lower() or "fixed" in text.lower() or "commited" in text.lower() or "github" in text.lower() or "overall" in text.lower() else 0)

In [106]:
jira_issues['Label_extract'] = jira_issues['Labels_complete'].apply(lambda text: 1 if "pull" in str(text).lower() or "fix" in str(text).lower() or "issue" in str(text).lower() else 0)

In [107]:
jira_issues['Final_extract']  = np.logical_or.reduce((jira_issues['Log_text_extract'],jira_issues['attachment_extract'],jira_issues['comments_extract'],jira_issues['Label_extract']))

In [108]:
jira_issues['Final_extract'].value_counts()

True     14937
False     5857
Name: Final_extract, dtype: int64

In [144]:
import itertools
drop_cols = list(itertools.chain(log_columns, comments_columns, attach_columns))

In [146]:
jira_issues.drop(drop_cols, axis=1, inplace=True)

In [151]:
jira_issues.drop(['Log_Text_Complete','attach_complete','comments_complete','Labels_complete'], axis=1, inplace=True)

In [156]:
jira_issues.to_csv("Jira_issues_261021.csv", line_terminator='\n')

## Done till Here

In [157]:
## Extract stacktrace if exist

In [2]:
jira_issues = pd.read_csv("Jira_issues_261021.csv")

C:\Users\suresh.nagulavancha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
jira_issues['code_exist']=jira_issues['Description'].apply(lambda x:1 if "{code" in str(x) else 0)

In [105]:
def extract_code(text):
    codes = ""
    split_text = str(text).split("{code")
    for i,te in enumerate(split_text):
        if i%2==1:
            codes+="}".join(te.split("}")[1:]).strip()
    return codes

def remove_code(text):
    description = ""
    split_text = str(text).split("{code")
    if len(split_text)==1:
        return split_text[0]
    for i,te in enumerate(split_text):
        if i%2==0:
            description+="}".join(te.split("}")[1:]).strip()
    return description

In [106]:
jira_issues['codes'] = jira_issues['Description'].apply(extract_code)
jira_issues['Description_code_less'] = jira_issues['Description'].apply(remove_code)

In [110]:
jira_issues.to_csv("Jira_issues_261021.csv", line_terminator='\n', index=False)

In [111]:
## Done till here

In [4]:
#TF-IDF, Word2vec, Spacy

In [5]:
## TF-IDF Data

In [6]:
jira_issues = pd.read_csv("jira_issues_261021.csv")

C:\Users\suresh.nagulavancha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
## TF-IDF Data

In [9]:
jira_issues_tfidf = jira_issues.copy()

In [97]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tqdm import tqdm

In [69]:
sw = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
vectorizer = TfidfVectorizer()

In [63]:
jira_issues_tfidf.columns

Index(['Issue key', 'Summary', 'Issue Type', 'Status', 'Project key',
       'Project description', 'Resolution', 'Affects Version/s',
       'Affects Version/s.1', 'Affects Version/s.2', 'Affects Version/s.3',
       'Affects Version/s.4', 'Affects Version/s.5', 'Fix Version/s',
       'Fix Version/s.1', 'Fix Version/s.2', 'Fix Version/s.3',
       'Fix Version/s.4', 'Component/s', 'Component/s.1', 'Component/s.2',
       'Component/s.3', 'Labels', 'Labels.1', 'Description', 'Environment',
       'Affects Version/s.6', 'Affects Version/s.7', 'Labels.2', 'Labels.3',
       'Labels.4', 'Labels.5', 'Labels.6', 'Log_text_extract',
       'attachment_extract', 'comments_extract', 'Label_extract',
       'Final_extract', 'code_exist', 'codes', 'Description_code_less'],
      dtype='object')

In [64]:
cols_intrest = ['Summary','Issue Type','Component/s','Component/s','Labels','codes','Description_code_less','Description']

In [ ]:
## Lower, remove puncts, lemma, stopwords, num2words

In [66]:
def remove_stopwords_do_lemma(text):
    text = str(text)
    text_tokens = word_tokenize(text)
    text_filtered = [lemmatizer.lemmatize(word) for word in text_tokens if not word in sw]
    return " ".join(text_filtered)

In [67]:
symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n\r"
for col in cols_intrest:
    if col=='codes':
        continue
    else:
        jira_issues_tfidf[col] = jira_issues_tfidf[col].str.lower()
        jira_issues_tfidf[col] = jira_issues_tfidf[col].apply(remove_stopwords_do_lemma)
        jira_issues_tfidf[col] = jira_issues_tfidf[col].apply(lambda x:"".join([char for char in x if char not in symbols]))
        

In [87]:
def tokenize_text(text):
    tokens = []
    for sent in sent_tokenize(text):
        for word in word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [88]:
jira_issues_summary_docvec = jira_issues_tfidf.apply(lambda r: TaggedDocument(words=tokenize_text(r['Summary']), tags=[r['Project key']]), axis=1)

In [93]:
jira_issues_description_doc2vec = jira_issues_tfidf.apply(lambda r: TaggedDocument(words=tokenize_text(r['Description_code_less']), tags=[r['Project key']]), axis=1)
jira_issues_descriptionwcode_doc2vec = jira_issues_tfidf.apply(lambda r:TaggedDocument(words=tokenize_text(r['Description']), tags=[r['Project key']]), axis=1)

In [90]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [123]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(jira_issues_summary_docvec.values)])
model_dbow_desc = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow_desc.build_vocab([x for x in tqdm(jira_issues_description_doc2vec.values)])
model_dbow_wcode = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow_wcode.build_vocab([x for x in tqdm(jira_issues_descriptionwcode_doc2vec.values)])

100%|███████████████████████████████████████████████████████████████████████| 20794/20794 [00:00<00:00, 2577010.91it/s]


In [124]:
for epoch in range(30):
    model_dbow.train(jira_issues_summary_docvec, total_examples=len(jira_issues_summary_docvec.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha
    model_dbow_desc.train(jira_issues_description_doc2vec, total_examples=len(jira_issues_description_doc2vec.values), epochs=1)
    model_dbow_desc.alpha -= 0.002
    model_dbow_desc.min_alpha = model_dbow_desc.alpha
    model_dbow_wcode.train(jira_issues_descriptionwcode_doc2vec, total_examples=len(jira_issues_descriptionwcode_doc2vec.values), epochs=1)
    model_dbow_wcode.alpha -= 0.002
    model_dbow_wcode.min_alpha = model_dbow_wcode.alpha

In [126]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return regressors

In [127]:
summary_df = pd.DataFrame(vec_for_learning(model_dbow, jira_issues_summary_docvec))
desc_df = pd.DataFrame(vec_for_learning(model_dbow_desc, jira_issues_description_doc2vec))
desc_wcode_df = pd.DataFrame(vec_for_learning(model_dbow_wcode, jira_issues_descriptionwcode_doc2vec))

In [129]:
summary_df.columns = ["summary_"+str(i) for i in range(300)]
desc_df.columns = ["description_"+str(i) for i in range(300)]
desc_wcode_df.columns = ["wcode_"+str(i) for i in range(300)]

In [154]:
jira_issues['Final_extract'] = jira_issues['Final_extract'].astype(int)

In [155]:
dataset_build_target = jira_issues[['Issue key','Issue Type','Project key','Final_extract']]

In [158]:
dataset_build = pd.concat([dataset_build_target,summary_df, desc_df, desc_wcode_df], axis=1)

In [160]:
dataset_build.to_csv("Final_data_defect_classification.csv", line_terminator='\n', index=False)